In [ ]:
!pip install colorama
!pip install scikit_learn
!pip install Flask
!pip install beautifulsoup4
!pip install bs4
!pip install googletrans==4.0.0rc1

In [26]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import googletrans
from googletrans import Translator
import requests
import pandas as pd


In [27]:
###########
# This is the almost complete project. Run this in order to run tests after.
###########


import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle
translator = Translator()

data = pd.read_json("https://raw.githubusercontent.com/lucdekeijzer/film_recommendations/main/intents.json")
 

training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)


lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

# to save the trained model
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)
    
    
def GetTitlesFromSoup(results):                                    
    titles = []
    for word in results:
        for j in word:
            for i in j:
                for p in i:
                    is_year_of_pub = True                          #filtering out years of publication; filters everything that consists only of numbers, so a movie like 2012 will not make it to the recommendation
                    for char in p:
                        if ord(char) < 48 or ord(char) > 57:
                            is_year_of_pub = False
                            break
                    if is_year_of_pub == False:                    #Only if p is not a string of just numbers, it is put in the titles list
                        titles.append(p)
    return titles




def webscraper(x):
  from urllib.request import Request, urlopen
  from bs4 import BeautifulSoup
  import requests
  from random import randint

  #x = inp

  root = "https://www.google.com"
  link = "https://www.google.com/search?q=top+10+{}+movies&oq=top+&aqs=chrome.0.69i59l3j69i57j69i60l4.651j0j4&sourceid=chrome&ie=UTF-8".format(x)
  results = []

  req = Request(link, headers = {"User-Agent": "Mozilla/5.0"})
  webpage = urlopen(req).read()
  with requests.Session() as c:
    soup = BeautifulSoup(webpage, "html5lib")
    #print(soup)
    for item in soup.find_all('div', attrs = {"class": "RWuggc"}):
      results.append(item)

  titles = GetTitlesFromSoup(results)
  
  end_of_titles = len(titles)
  movie_index = randint(0, end_of_titles )
  movie_index_end = movie_index + 1
  return str(*(titles[movie_index:movie_index_end]))



def chat():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)

    # parameters
    max_len = 20

    language_detect = input("Please state the language you would like to talk in to Moviebot? \n (State the language in English please) ")
    language_detect = language_detect.lower()
    language_lib = googletrans.LANGCODES

    
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        #language = (translator.detect(inp)).lang
        language = language_lib[language_detect]
        inp_translated = (translator.translate(inp)).text
        if inp_translated.lower() == "quit":
            break
        


        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp_translated]),
                                             truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
          if i['tag'] == tag:
            answer = np.random.choice(i['responses'])
            trans_answer = translator.translate(answer, dest = language).text
            print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)

        if tag == "thanks":
            answer = np.random.choice(i['responses'])
            trans_answer = translator.translate(answer, dest = language).text
            #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
            break
        if tag == "genres":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(translator.translate("What about: ", dest = language).text, webscraper(inp_translated))
          genre = inp_translated
        if tag == "another":
          answer = np.random.choice(i['responses'])
          trans_answer = translator.translate(answer, dest = language).text
          #print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL , trans_answer)
          print(webscraper(genre))
        
          
          






Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_1   (None, 16)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 16)                272       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 10)                170       
                                                                 
Total params: 16,714
Trainable params: 16,714
Non-trainable params: 0
__________________________________________________

In [34]:
#Testing
#1. Unit Testing
import pytest

def LangDetectTest():
  correct_input = "Lorem Ipsum"
  incorrect_input = ""

  outcome1 = (translator.detect(correct_input)).lang
  if outcome1 == "la":
    test1 = "Success"
  else:
    test1 = "Failure"
  try:
    outcome2 = (translator.detect(incorrect_input)).lang
    test2 = "Success"
  except:
    test2 = "Failure"
    return "Result of language detector test with correct input: {}\n\
Result of language detector test with incorrect input: {} caused by ERROR".format(test1, test2)
  else:
    return "Result of language detector test with correct input: {};\n\
Result of language detector test with incorrect input: {}".format(test1, test2)


def TranslateTest():
    correct_input = "Hello, how are you doing?"
    incorrect_input = "eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee\
    eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee"

    desired_outcome1 = "Hallo hoe is het?"
    
    outcome1 = translator.translate(correct_input, dest = "nl").text

    if outcome1 == desired_outcome1:
        test1 = "Success"
    else:
        test1 = "Failure"
    try:
        outcome2 = translator.translate(incorrect_input, dest = "en").text
        test2 = "Success"
    except:
        test2 = "Failure"
        return "Result of language detector test with correct input: {}\n\
Result of language detector test with incorrect input: {} caused by ERROR".format(test1, test2)
    else:
        return "Result of language detector test with correct input: {}\n\
Result of language detector test with incorrect input: {}".format(test1, test2)


def LoadModelTest():
    try:
        model = keras.models.load_model('chat_model')
        test1 = "Success"
    except:
        test1 = "Failure"
    return "Loading the model resulted in: {}".format(test1)
        
    
def TitleFilterTest():
  from urllib.request import Request, urlopen
  from bs4 import BeautifulSoup
  import requests
  
  #Generating test input
  
  #correct_input = ['<div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Raid: Redemption</div></div><div><div class="BNeawe tAd8D AP7Wnd">2011</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Inception</div></div><div><div class="BNeawe tAd8D AP7Wnd">2010</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Casino Royale</div></div><div><div class="BNeawe tAd8D AP7Wnd">2006</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Matrix</div></div><div><div class="BNeawe tAd8D AP7Wnd">1999</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">John Wick</div></div><div><div class="BNeawe tAd8D AP7Wnd">2014</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Dark Knight</div></div><div><div class="BNeawe tAd8D AP7Wnd">2008</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">F9</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Die Hard</div></div><div><div class="BNeawe tAd8D AP7Wnd">1988</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Free Guy</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">No Time To Die</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Black Widow</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Tenet</div></div><div><div class="BNeawe tAd8D AP7Wnd">2020</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Extraction</div></div><div><div class="BNeawe tAd8D AP7Wnd">2020</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Mission: Impossible...</div></div><div><div class="BNeawe tAd8D AP7Wnd">2018</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Edge Of Tomorrow</div></div><div><div class="BNeawe tAd8D AP7Wnd">2014</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Atomic Blonde</div></div><div><div class="BNeawe tAd8D AP7Wnd">2017</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Old Guard</div></div><div><div class="BNeawe tAd8D AP7Wnd">2020</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Suicide Squad</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Skyfall</div></div><div><div class="BNeawe tAd8D AP7Wnd">2012</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Shang-Chi and the Le...</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Bourne Identity</div></div><div><div class="BNeawe tAd8D AP7Wnd">2002</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Avengers: Endgame</div></div><div><div class="BNeawe tAd8D AP7Wnd">2019</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Kill Bill: Vol. 1</div></div><div><div class="BNeawe tAd8D AP7Wnd">2003</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Raiders Of The Lost Ark</div></div><div><div class="BNeawe tAd8D AP7Wnd">1981</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Venom: Let There Be...</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Terminator 2: Judgme...</div></div><div><div class="BNeawe tAd8D AP7Wnd">1991</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Gladiator</div></div><div><div class="BNeawe tAd8D AP7Wnd">2000</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Mad Max 2: The Road...</div></div><div><div class="BNeawe tAd8D AP7Wnd">1981</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Aliens</div></div><div><div class="BNeawe tAd8D AP7Wnd">1986</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Point Break</div></div><div><div class="BNeawe tAd8D AP7Wnd">1991</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Speed</div></div><div><div class="BNeawe tAd8D AP7Wnd">1994</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Bourne Ultimatum</div></div><div><div class="BNeawe tAd8D AP7Wnd">2007</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Army of the Dead</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Eternals</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Raid 2</div></div><div><div class="BNeawe tAd8D AP7Wnd">2014</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Black Panther</div></div><div><div class="BNeawe tAd8D AP7Wnd">2018</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">THE FUGITIVE...</div></div><div><div class="BNeawe tAd8D AP7Wnd">1993</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Spider-Man : No Way...</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Taken</div></div><div><div class="BNeawe tAd8D AP7Wnd">2008</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Terminator</div></div><div><div class="BNeawe tAd8D AP7Wnd">1984</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Spectre</div></div><div><div class="BNeawe tAd8D AP7Wnd">2015</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">First Blood</div></div><div><div class="BNeawe tAd8D AP7Wnd">1982</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Avengers</div></div><div><div class="BNeawe tAd8D AP7Wnd">2012</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Hard Boiled</div></div><div><div class="BNeawe tAd8D AP7Wnd">1992</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Avengers: Infinity War</div></div><div><div class="BNeawe tAd8D AP7Wnd">2018</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Enter the Dragon</div></div><div><div class="BNeawe tAd8D AP7Wnd">1973</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Red Notice</div></div><div><div class="BNeawe tAd8D AP7Wnd">2021</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Lethal Weapon</div></div><div><div class="BNeawe tAd8D AP7Wnd">1989</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Ip Man</div></div><div><div class="BNeawe tAd8D AP7Wnd">2008</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Star Wars: Episode IV...</div></div><div><div class="BNeawe tAd8D AP7Wnd">1977</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">RoboCop</div></div><div><div class="BNeawe tAd8D AP7Wnd">1987</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Major Grom: Plague Do...</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Mowgli: Legend of...</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">The Witcher: N...</div></div></div>, <div class="RWuggc kCrYT"><div><div class="BNeawe s3v9rd AP7Wnd">Double World</div></div></div>']
  #First tried the above input^^, was not the right format.
  root = "https://www.google.com"
  link = "https://www.google.com/search?q=top+10+{}+movies&oq=top+&aqs=chrome.0.69i59l3j69i57j69i60l4.651j0j4&sourceid=chrome&ie=UTF-8".format("action")
  results = []

  req = Request(link, headers = {"User-Agent": "Mozilla/5.0"})
  webpage = urlopen(req).read()
  with requests.Session() as c:
    soup = BeautifulSoup(webpage, "html5lib")
    #print(soup)
    for item in soup.find_all('div', attrs = {"class": "RWuggc"}):
      results.append(item)
    
  #Test

  desired_outcome1 = ['The Matrix', 'Die Hard', 'The Dark Knight', 'John Wick', 'Casino Royale', 'Terminator 2: Judgme...', 'Aliens', 'No Time To Die', 'Black Widow', 'Skyfall', 'Avengers: Endgame', 'Mission: Impossible...', 'Edge Of Tomorrow', 'Army of the Dead', 'Extraction', 'Free Guy', 'Shang-Chi and the Le...', 'Kill Bill: Vol. 1', 'The Suicide Squad', 'Spectre', 'Raiders Of The Lost Ark', 'The Terminator', 'Gladiator', 'The Old Guard', 'Jungle Cruise', 'Venom: Let There Be...', 'Nobody', 'Tenet', 'First Blood', 'F9', 'Predator', 'Eternals', 'Black Panther', 'V for Vendetta', 'Mad Max: Fury Road', 'Lethal Weapon', '21 Jump Street', 'Enter the Dragon', 'Inception', 'Logan', 'Red Notice', 'The Bourne Identity', 'Birds of Prey: Harl...', 'Taken', 'Mortal Kombat', 'Venom', 'Total Recall', 'Sooryavanshi', 'Thor: Ragnarok', 'Guardians of the Gala...', 'Speed', 'Monster Hunter', 'Tenet', 'The Old Guard', 'Extraction']

  test1 = "Failure"

  try:
    outcome1 = GetTitlesFromSoup(results)
    

  except:
    test1 = "Failure"
    return "Result of Title Filter Test with correct input: {} caused by ERROR".format(test1)

  else:
    for title in outcome1:
      if title in desired_outcome1:
        test1 = "Success"
        break
    return "Result of Title Filter Test with correct input: {}".format(test1)

    
    
    


#2. Component/Integration testing

def WebscraperTest():
  known_genre = "action"
  unknown_genre = "bruh"

  outcome1 = webscraper(known_genre)
  if type(outcome1) == str and len(outcome1) > 0:
    test1 = "Success"
  else:
    test1 = "Failure"
  try:
    outcome2 = webscraper(unknown_genre)
    test2 = "Success"
  except:
    test2 = "Failure"
    return "Result of webscraper test with a known genre as input: {}\n\
    Result of webscraper test with an unknown genre as input: {} caused by ERROR".format(test1, test2)
  else:
    return "Result of webscraper test with a known genre as input: {}\n\
Result of webscraper test with an unknown genre as input: {}".format(test1, test2)

def intentTest():
    def getOutput(input):
        # load trained model
        model = keras.models.load_model('chat_model')
        # load tokenizer object
        with open('tokenizer.pickle', 'rb') as handle:
            tokenizer = pickle.load(handle)
        # load label encoder object
        with open('label_encoder.pickle', 'rb') as enc:
            lbl_encoder = pickle.load(enc)
        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([input]),truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])

        for i in data['intents']:
            if i['tag'] == tag:
                output = np.random.choice(i['responses'])
        return output
    
    ## TEST CASES ##
    correct = "What can I do here?"
    misspelled = "wat can i do"
    expected_output = ["I'm here to find movies for you.", "You can ask me to find a movie for you.", "If you ask me for a movie I will fetch one for you.", 
    "I can help you find movies. Just ask me if you want to find a movie.", "I find movies. What genre of movies do you like?"]

    ## TEST ## 
    if getOutput(correct) in expected_output:
        result1 = "Result of intent test with correct input ({}): Success".format(correct)
    else:
        result1 = "Result of intent test with correct input ({}): Failure".format(correct)

    if getOutput(misspelled) in expected_output:
        result2 = "Result of intent test with misspelled input ({}): Success".format(misspelled)
    else:
        result2 = "Result of intent test with misspelled input ({}): Failure".format(misspelled)
    return str(result1 + '\n' + result2)



In [35]:


#Unit tests
print(LangDetectTest())

print(TranslateTest())

print(LoadModelTest())

print(TitleFilterTest())

print("\nUnit testing finished\n")

#Component tests
print(WebscraperTest())

print(intentTest())

print("\nComponent testing finished")

Result of language detector test with correct input: Success
Result of language detector test with incorrect input: Failure caused by ERROR
Result of language detector test with correct input: Success
Result of language detector test with incorrect input: Success
Loading the model resulted in: Success
Result of Title Filter Test with correct input: Failure

Unit testing finished

Result of webscraper test with a known genre as input: Failure
Result of webscraper test with an unknown genre as input: Success
Result of intent test with correct input (What can I do here?): Success
Result of intent test with misspelled input (wat can i do): Failure

Component testing finished
